# Bokeh Dashboard

In [25]:
from bokeh.io import output_notebook

In [7]:
output_notebook()

Loading BokehJS ...

## Basic line

In [87]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh_sampledata.degrees import data

source = ColumnDataSource(data=data)

p_line = figure(title="Bachelor’s degrees in Engineering earned by women over the years", x_axis_label="Year", y_axis_label="Number of degrees")
p_line.line(x="Year", y="Engineering", source=source)

show(p_line)

## Multiple lines, tooltips, mute legend

In [104]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh_sampledata.degrees import data

source = ColumnDataSource(data=data)

TOOLTIPS = [
    ("Year", "@Year"),
    ("No.of degrees", "$y"),
     ]

p_mline = figure(
    title="Bachelor’s degrees in Engineering earned by women over the years",
    x_axis_label="Year", 
    y_axis_label="Number of degrees",
    tooltips=TOOLTIPS
)

p_mline.line(x="Year", y="Engineering", source=source, legend_label="Engineering")
p_mline.line(x="Year", y="Business", source=source, legend_label="Business", line_dash="dashed")

p_mline.legend.click_policy="mute"
p.legend.nrows=3

show(p_mline)

## Bars with span

In [120]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Span
from bokeh_sampledata.degrees import data

df_2010 = data[data["Year"] == 2010]
df_2010 = df_2010.T.drop("Year")

mean = int(df_2010.mean().iloc[0])
median = int(df_2010.median().iloc[0])

p_bar = figure(
    title="Bachelor's Degrees earned by women in 2010 across fields", 
    x_axis_label="Fields", 
    y_axis_label="Number of degrees",
    x_range=list(df_2010.index),
)

p_bar.vbar(x=list(df_2010.index), top=df_2010[40])

p_bar.xaxis.major_label_orientation = 1

span_mean = Span(location=mean, dimension='width')
p_bar.add_layout(span_mean)

span_median = Span(location=median, dimension='width')
p_bar.add_layout(span_median)

show(p_bar)

## Scatter with legend annotations

In [126]:
from bokeh.plotting import figure, show
from bokeh.sampledata.penguins import data
from bokeh.transform import factor_cmap, factor_mark

SPECIES = sorted(data.species.unique())
MARKERS = ['hex', 'circle_x', 'triangle']

p_scatter = figure(title = "Penguin size", background_fill_color="#fafafa")
p_scatter.xaxis.axis_label = 'Flipper Length (mm)'
p_scatter.yaxis.axis_label = 'Body Mass (g)'

p_scatter.scatter("flipper_length_mm", "body_mass_g", source=data,
          legend_group="species", fill_alpha=0.4, size=12,
          marker=factor_mark('species', MARKERS, SPECIES),
          color=factor_cmap('species', 'Category10_3', SPECIES))

p_scatter.legend.location = "top_left"
p_scatter.legend.title = "Species"
p_scatter.legend.ncols=3

show(p_scatter)

## Dashboard with layout

In [127]:
from bokeh.layouts import layout
from bokeh.plotting import show, output_file

dashboard = layout([
    [p_line, p_mline],
    [p_bar],
    [p_scatter],
])

show(dashboard)

In [32]:
p4 = figure(title="Line & Scatter", x_axis_label="x", y_axis_label="y")

p4.line(x=x, y=y1)
p4.scatter(x=x, y=y1)

show(p4)